In [44]:
import pandas as pd
import numpy as np

In [45]:
df_2020 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\BASE_YEAR\2020_jtmt_forcast_full_240506.xlsx").set_index('Taz_num').sort_index() 
df_jtmt_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_jtmt.xlsx").set_index('Taz_num').sort_index() 
df_iplan_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_iplan.xlsx").set_index('Taz_num').sort_index() 
df_bau_2050 = pd.read_excel(r"C:\Users\dpere\Documents\JTMT\forecast_by_version\V4\240506_forecast_2020_till_2050_bau.xlsx").set_index('Taz_num').sort_index() 

In [46]:
def add_total_by_type_row(output_df, column_input_name, column_input_value, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df[df[f'{column_input_name}'] == column_input_value] 
        values_list.append(len(val))

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [47]:
def add_total_row(output_df, column_input_name, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if column_input_name in df.columns:
            val = df.pivot_table(index='main_secto', values=f'{column_input_name}', aggfunc="sum", fill_value=0).astype('float32').sum()
            values_list.append("{:,}".format(int(val.iloc[0])))
        else:
            values_list.append('NaN')

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [48]:
def add_taz_num_is_unique_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = df_2020.index.is_unique
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [49]:
def add_taz_num_count_row(output_df, column_output_name):
    values_list = []
    
    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        val = len(df.index)
        values_list.append(val)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [50]:
def add_percentage_growth_row(current, previous, output_df, column_output_name):
    values_list = []

    for idx, df in enumerate([df_2020, df_jtmt_2050, df_iplan_2050, df_bau_2050]):
        if len(list(df)) == 135:
                values_list.append('NaN')

        else:
            if df[f'{current}'].sum() == df[f'{previous}'].sum():
                values_list.append(100.0)
            try:
                values_list.append(int((df[f'{current}'].sum() / df[f'{previous}'].sum()) * 100.0))
            except ZeroDivisionError:
                values_list.append(0)

    output_df.loc[f'{column_output_name}'] = values_list

    return output_df

In [51]:
monitoring_df = pd.DataFrame(columns=['2020', 'JTMT', 'iplan', 'bau'])

In [52]:
# אוכלוסיה
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva', 'pop_2020')
monitoring_df = add_total_row(monitoring_df, 'pop_without_dorms_yeshiva_2050', 'pop_2050')

# מועסקים
monitoring_df = add_total_row(monitoring_df, 'total_emp', 'emp_2020')
monitoring_df = add_total_row(monitoring_df, 'total_emp_2050', 'emp_2050')

# מספר א"ת לפי סקטור
monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', 'U_Orthodox', 'Taz_U_Orthodox')
monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', 'Palestinian', 'Taz_Palestinian')
monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', 'arabs_behined_seperation_wall', 'Taz_arabs_behined_seperation_wall')
monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', 'Arab', 'Taz_Arab')
monitoring_df = add_total_by_type_row(monitoring_df, 'main_secto', 'Jewish', 'Taz_Jewish')

# סטודנטים
monitoring_df = add_total_row(monitoring_df, 'univ', 'univ_2020')
monitoring_df = add_total_row(monitoring_df, 'univ_2050', 'univ_2050')

# בתי ספר
monitoring_df = add_total_row(monitoring_df, 'student', 'student_2020')
monitoring_df = add_total_row(monitoring_df, 'student_2050', 'student_2050')

# ישיבות
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim', 'student_yeshiva_and_kollim_2020')
monitoring_df = add_total_row(monitoring_df, 'student_yeshiva_and_kollim_2050', 'student_yeshiva_and_kollim_2050')

# zonetype
monitoring_df = add_total_by_type_row(monitoring_df, 'zonetype', 'Jerusalem', 'zonetype_Jerusalem')
monitoring_df = add_total_by_type_row(monitoring_df, 'zonetype', 'Judea and Samaria', 'zonetype_Judea and Samaria')
monitoring_df = add_total_by_type_row(monitoring_df, 'zonetype', 'Ramla', 'zonetype_Ramla')

# jeru_metro
monitoring_df = add_total_by_type_row(monitoring_df, 'jeru_metro', 0, 'jeru_metro_0')
monitoring_df = add_total_by_type_row(monitoring_df, 'jeru_metro', 1, 'jeru_metro_1')

# in_jerusal
monitoring_df = add_total_by_type_row(monitoring_df, 'in_jerusal', 'yes', 'in_jerusal_yes')
monitoring_df = add_total_by_type_row(monitoring_df, 'in_jerusal', 'no', 'in_jerusal_no')

# אין א"ת שחוזר פעמיים
monitoring_df = add_taz_num_is_unique_row(monitoring_df, 'taz_num_is_unique')
# מספר א"ת
monitoring_df = add_taz_num_count_row(monitoring_df, 'taz_num_count')

In [53]:
# גידול כל חומש בכל תרחיש
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2025', 'pop_without_dorms_yeshiva', monitoring_df, 'percentage growth 2020-2025')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2030', 'pop_without_dorms_yeshiva_2025', monitoring_df, 'percentage growth 2025-2030')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2035', 'pop_without_dorms_yeshiva_2030', monitoring_df, 'percentage growth 2030-2035')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2040', 'pop_without_dorms_yeshiva_2035', monitoring_df, 'percentage growth 2035-2040')
monitoring_df = add_percentage_growth_row('pop_without_dorms_yeshiva_2045', 'pop_without_dorms_yeshiva_2050', monitoring_df, 'percentage growth 2045-2050')

In [54]:
# monitoring_df.to_excel("monitoring_df.xlsx")
monitoring_df

,2020,JTMT,iplan,bau
pop_2020,"1,758,461","1,758,461","1,758,461","1,758,461"
pop_2050,NaN,"7,555,735","7,278,700","7,759,649"
emp_2020,"482,716","482,716","482,716","482,716"
emp_2050,NaN,"989,702","837,076","962,301"
Taz_U_Orthodox,174,174,184,206
Taz_Palestinian,157,157,157,157
Taz_arabs_behined_seperation_wall,5,5,5,5
Taz_Arab,86,86,86,86
Taz_Jewish,482,482,472,450
univ_2020,"81,184","81,184","81,184","81,184"
